# Esercizio 6

Dato un file in formato `GTF` (Gene Transfer Format) che annota un set di geni sulla stessa genomica di riferimento, e il file della genomica di riferimento (*genomic reference*) in formato `FASTA`, produrre in output:

- l'elenco degli esoni annotati, ciascuno con il set dei trascritti di appartenenza (e il relativo gene). Gli esoni devono essere elencati per numero crescente di trascritti di appartenenza
- l'elenco degli esoni completamente coperti da coding sequence, specificandone la suddivisione in codoni in relazione ad ognuno dei trascritti di appartenenza

***

Input:

- nome del file in formato `GTF`
- nome del file della sequenza genomica in formato `FASTA`

***

Requisiti:

- deve essere definita una funzione `reverse_complement_in_case()` che prenda come argomento una sequenza nucleotidica e un valore di *strand* e ne restituisca il reverse&complement se lo strand è `-`, altrimenti restituisce la sequenza così com'è
- deve essere definita una funzione `codon_splitting()` che prenda come argomento una sequenza nucleotidica e un valore di *frame* in `{0, 1, 2}`, operi la suddivisione in codoni della sequenza (tenendo conto del valore di *frame*) e restituisca una stringa che unisca i codoni usando il carattere di spazio come separatore
- deve essere definita una funzione `get_transcript_and_gene()` che prenda come argomento una record `GTF` e restituisca una tupla contenente l'ID del trascritto come primo elemento e l'ID del gene come secondo elemento.


**NOTA BENE**: gli attributi (coppie *nome-valore*) del nono campo del file GTF non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `annotated_exon_list`: lista degli esoni annotati (ordinati per numero decrescente di trascritti di appartenenza); ogni elemento è una lista annidata di due elementi: tupla (*start*, *end*) dell'esone e `set` dei trascritti a cui l'esone appartiene (ogni trascritto deve essere rappresentato tramite la tupla *(transcript_id, gene_id)*)
- `exon_coverage_list`: lista degli esoni coperti interamente da coding sequence con la relativa suddivisione in codoni; ogni elemento della lista è una tupla dei seguenti cinque elementi: *transcript_id*, *gene_id*, start dell'esone, end dell'esone, suddivisione in codoni.

**NOTA BENE**: uno esone può comparire in elementi diversi della lista `exon_coverage_list` dal momento che può essere incluso in trascritti diversi.

***

### Note sul formato `GTF`

#### *Feature* e *record* `GTF`

Una *feature* `GTF` è un intervallo di posizioni sulla genomica di riferimento che ha un certo significato funzionale, ad esempio un esone (*feature* di tipo `exon`) o un frammento della coding sequence di un trascritto (*feature* di tipo `CDS`). 

Un *record* `GTF` rappresenta una *feature* di un certo tipo inclusa in un determinato trascritto (di un determinato gene). Il tipo di *feature* è specificato nel terzo campo del *record*.

Ad esempio il *record*:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    
rappresenta un esone (*feature* di tipo `exon`) che inizia in posizione `64566` e finisce in posizione `64757`, incluso nel trascritto `U52112.4-014` del gene `ARHGAP4`.

Invece il *record*:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence (*feature* di tipo `CDS`) del trascritto `U52112.4-005` del gene `ARHGAP4`, mappato sulla genomica di riferimento dalla posizione `70312` alla posizione `70440`.

#### Esone incluso in trascritti diversi

Tutti i *record* di tipo `exon`, che corrispondono alla stessa *feature* sulla genomica di riferimento (cioé allo stesso intervallo di posizioni) rappresentano lo stesso esone incluso in trascritti diversi.

Ad esempio i sei *record* seguenti:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-002"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-003"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-001"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-024"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-011"; gene_id "ARHGAP4";
    
rappresentano l'esone `[64566, 64757]` incluso in sei trascritti diversi del gene `ARHGAP4`.

#### Esone coperto da coding sequence per un dato trascritto

Un esone di un certo trascritto è coperto completamente da coding sequence se, oltre al *record* di tipo `exon` che rappresenta l'esone incluso nel trascritto, esiste anche un *record* di tipo `CDS` corrispondente allo stesso intervallo di posizioni.

Ad esempio i due *record*:

    ENm006 VEGA_Known exon	70312	70440	.	-	.	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

indicano che l'esone `[70312, 70440]` (incluso nel trascritto `U52112.4-005`del gene `ARHGAP4`) è coperto completamente da coding sequence.

#### Suddivisione in codoni di una *feature* di tipo `CDS`

La suddivisione in codoni di una *feature* di tipo `CDS` deve tenere conto del valore del campo *frame* (ottavo campo del *record*), che specifica la posizione della prima base della *feature* all'interno del codone di appartenenza.

Ad esempio il *record* di tipo `CDS` seguente:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    
rappresenta un frammento della coding sequence del trascritto `U52112.4-005` del gene `ARHGAP4`. Il valore 0 del campo *frame* indica che la prima base della sequenza della *feature* è la prima base di un codone (cioé le prime tre basi della *feature* sono un codone completo).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      
 
    cggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggac

la sua suddivisione in codoni sarà dunque:

    cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac

Il *record* di tipo `CDS` seguente:
    
    ENm006 VEGA_Known CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence del trascritto `U52112.4-019` del gene `ARHGAP4`. Il valore 1 del campo *frame* indica che la prima base della sequenza della *feature* è la seconda base di un codone (cioé le prime due basi della *feature* sono le ultime due basi di un codone la cui prima base sarà l'ultima della *feature* `CDS` precedente).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      

    gaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaag

la sua suddivisione in codoni sarà dunque:

    ga agg agc cgt ccc tcc tgt cgc cct tgc act gct ggg cgg tgc tgc tgc agc aca cgc ggc agc aga gcc ggg aga gcg cgg ccc tga gtg agg tgc tgg ccg ggc ccc tgg ccc agc gcc tga gtc aca ttg cag agg acg tgg ggc gcc tgg tca aga ag

Il *record* di tipo `CDS` seguente:

    ENm006 VEGA_Known CDS	72761	72965	.	-	2	transcript_id "U52112.4-003"; gene_id "ARHGAP4";

rappresenta un frammento della coding sequence del trascritto `U52112.4-003` del gene `ARHGAP4`. Il valore 2 del campo *frame* indica che la prima base della sequenza della *feature* è la terza base di un codone (cioé la prima base della *feature* è l'ultima base di un codone le cui prime due basi saranno le ultime della *feature* `CDS` precedente).
Tenendo presente che la sequenza della *feature* estratta dalla *genomic reference* è:                      

    agatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccg

la sua suddivisione in codoni sarà dunque:

    a gat gcg ctg gca gct gag cga gca gct gcg ctg cct gga gct gca ggg cga gct gcg gcg gga gtt gct gca gga gct ggc aga gtt cat gcg gcg ccg cgc tga ggt gga gct gga ata ctc ccg ggg cct gga aaa gct ggc cga gcg ctt ctc cag ccg tgg agg ccg cct ggg gag cag ccg gga gca cca aag ctt ccg

***

## Soluzione

Importare il modulo `re` per usare le espressioni regolari.

In [1]:
import re

### Definizione della funzione `reverse_complement_in_case()`

La funzione deve prendere come argomento una sequenza nucleotidica e un valore di *strand* e deve restituire il reverse&complement se lo strand è `-`, altrimenti restituisce la sequenza così com'è.

In [2]:
def reverse_complement_in_case(nucleotide_sequence, strand):
    if strand == '-':
        complement_dict = {'a':'t', 't':'a', 'c': 'g', 'g':'c'}
        reverse_list = re.findall('\w', nucleotide_sequence.lower())[::-1]
        return ''.join(complement_dict[c] for c in reverse_list)
    else:
        return nucleotide_sequence.lower()

**NOTA BENE**: fare in modo che la funzione restituisca sempre una versione della sequenza in minuscolo.

### Definizione della funzione `codon_splitting()`

La funzione deve prendere come argomento una sequenza nucleotidica e un valore di *frame* in `{0, 1, 2}`, deve operare la suddivisione in codoni (tenendo conto del valore di *frame*) e restituire una stringa che unisca i codoni usando il carattere di spazio come separatore.

In [3]:
def codon_splitting(nucleotide_sequence, frame):
    return ' '.join([nucleotide_sequence[:3-frame]]+re.findall('\w{1,3}', nucleotide_sequence[3-frame:]))

### Definizione della funzione `get_transcript_and_gene()`

La funzione deve prendere come argomento un record `GTF` e deve restituire una tupla contenente l'ID del trascritto come primo elemento e l'ID del gene come secondo elemento.

In [4]:
def get_transcript_and_gene(row):
    transcript_id = re.findall('transcript_id\s+"([^"]+)";', row.rstrip().split('\t')[8])[0]
    gene_id = re.findall('gene_id\s+"([^"]+)";', row.rstrip().split('\t')[8])[0]
    return (transcript_id, gene_id)

### Parametri in input

In [5]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

### Lettura della genomica di riferimento

Lettura delle righe del file della genomica di riferimento nella lista `reference_file_rows`

In [6]:
with open(reference_file_name, 'r') as reference_input_file:
    reference_file_rows = reference_input_file.readlines()

In [7]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

Concatenazione delle righe contenenti la sequenza nucleotidica (dopo avere eliminato il simbolo di *newline* finale) nella variabile `genomic_reference`

In [8]:
genomic_reference = ''.join(row.rstrip() for row in reference_file_rows[1:])

In [9]:
genomic_reference

'...'

### Lettura dei *record* del file `GTF`

Lettura delle righe del file `GTF` nella lista `gtf_file_rows`

In [10]:
with open(gtf_file_name, 'r') as gtf_input_file:
    gtf_file_rows = gtf_input_file.readlines()

In [11]:
#gtf_file_rows

### Selezione dei *record* di tipo `exon` e di tipo `CDS`

Estrarre i *record* di tipo `exon` e i *record* di tipo `CDS` nelle due liste distinte `exon_gtf_rows` e `cds_gtf_rows`

In [12]:
exon_gtf_rows = [row for row in gtf_file_rows if row.rstrip().split('\t')[2] == 'exon']
cds_gtf_rows = [row for row in gtf_file_rows if row.rstrip().split('\t')[2] == 'CDS']

In [13]:
exon_gtf_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64385\t64459\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t79484\t79511\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72761\t72965\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72521\t72683\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72253\t72379\t.\t-\t.\ttranscript_id

In [14]:
cds_gtf_rows

['ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64385\t64459\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72761\t72963\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72521\t72683\t.\t-\t1\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72253\t72315\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71872\t71965\t.\t-\t0\ttranscript_id "U52112.4

### Costruzione della lista degli esoni annotati ordinati per numero descrescente di trascritti di appartenenza

A partire dalla lista `exon_gtf_rows` costruire:

il dizionario `exon_inclusion_dict`:
- *chiave*: tupla *(start, end)* che rappresenta un esone
- *valore*: `set` dei trascritti che includono l'esone (ogni trascritto deve essere rappresentato dalla tupla *(transcript_id, gene_id)* in modo da riferire il trascritto al gene di appartenenza)

Inizializzazione del dizionario vuoto.

In [15]:
exon_inclusion_dict = dict()

Attraversare la lista `exon_gtf_rows` per riempire il dizionario.

In [16]:
for row in exon_gtf_rows:
    (transcript_id, gene_id) = get_transcript_and_gene(row)
    exon_start = int(row.rstrip().split('\t')[3])
    exon_end = int(row.rstrip().split('\t')[4])
    
    exon_set = exon_inclusion_dict.get((exon_start,exon_end), set())
    exon_set.add((transcript_id, gene_id))
    exon_inclusion_dict.update([((exon_start,exon_end), exon_set)])

In [17]:
exon_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-004', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-015', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-012', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-012', 'ARHGAP4'),
  ('U52112.4-013', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005'

Ottenere dal dizionario `exon_inclusion_dict` la lista `annotated_exon_list` degli esoni annotati ordinati per numero crescente dei trascritti di appartenenza.

In [18]:
from collections import Counter

exon_counter = Counter(dict((key, len(exon_inclusion_dict[key])) for key in exon_inclusion_dict))
ordered_list = exon_counter.most_common()[::-1]

annotated_exon_list = [[exon[0], exon_inclusion_dict[exon[0]]] for exon in ordered_list]

In [19]:
annotated_exon_list

[[(57680, 58323), {('U52112.2-001', 'AVPR2')}],
 [(56689, 57573), {('U52112.2-001', 'AVPR2')}],
 [(55892, 55928), {('U52112.2-001', 'AVPR2')}],
 [(57680, 58322), {('U52112.2-002', 'AVPR2')}],
 [(57176, 57573), {('U52112.2-002', 'AVPR2')}],
 [(53688, 54049), {('U52112.2-002', 'AVPR2')}],
 [(56689, 57938), {('U52112.2-003', 'AVPR2')}],
 [(56271, 56327), {('U52112.2-003', 'AVPR2')}],
 [(542687, 542902), {('XX-FW83563B9.4-001', 'ATP6AP1')}],
 [(547709, 547769), {('XX-FW83563B9.4-006', 'ATP6AP1')}],
 [(542790, 542902), {('XX-FW83563B9.4-006', 'ATP6AP1')}],
 [(542894, 543223), {('XX-FW83563B9.4-004', 'ATP6AP1')}],
 [(546315, 546425), {('XX-FW83563B9.4-003', 'ATP6AP1')}],
 [(542694, 542902), {('XX-FW83563B9.4-003', 'ATP6AP1')}],
 [(543097, 545706), {('XX-FW83563B9.4-002', 'ATP6AP1')}],
 [(542747, 542902), {('XX-FW83563B9.4-002', 'ATP6AP1')}],
 [(64681, 64757), {('U52112.4-023', 'ARHGAP4')}],
 [(64935, 65410), {('U52112.4-023', 'ARHGAP4')}],
 [(70119, 70210), {('U52112.4-004', 'ARHGAP4')}],
 [

### Determinazione degli esoni coperti completamente da coding sequence

A partire dalla lista `exon_gtf_rows` costruire:

il dizionario `strand_dict`:
- *chiave*: ID del gene (hugo name)
- *valore*: *strand* del gene
    
Inizializzazione del dizionario vuoto.

In [20]:
strand_dict = dict()

Attraversare la lista `exon_gtf_rows` per riempire il dizionario.

In [21]:
for row in exon_gtf_rows:
    (transcript_id, gene_id) = get_transcript_and_gene(row)
    strand = row.rstrip().split('\t')[6]
    strand_dict[gene_id] = strand

In [22]:
strand_dict

{'ARHGAP4': '-', 'ATP6AP1': '+', 'AVPR2': '+'}

A partire dalla lista `cds_gtf_rows` costruire il dizionario `cds_inclusion_dict` delle *features* di tipo `CDS`:

- *chiave*: tupla *(start, end)* che rappresenta una *feature* di tipo `CDS`
- *valore*: set dei trascritti per cui la *feature* è un frammento di coding sequence (ogni trascritto deve essere rappresentato dalla tupla *(transcript_id, gene_id)* in modo da riferire il trascritto al gene di appartenenza)

Inizializzazione del dizionario vuoto.

In [23]:
cds_inclusion_dict = dict()

Attraversare la lista `cds_gtf_rows` per riempire il dizionario.

In [24]:
for row in cds_gtf_rows:
    (transcript_id, gene_id) = get_transcript_and_gene(row)
    cds_start = int(row.rstrip().split('\t')[3])
    cds_end = int(row.rstrip().split('\t')[4])
    
    cds_set = cds_inclusion_dict.get((cds_start,cds_end), set())
    cds_set.add((transcript_id, gene_id))
    cds_inclusion_dict.update([((cds_start,cds_end), cds_set)])

In [25]:
cds_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005', 'ARHGAP4')},
 (64385, 64459): {('U52112.4-005', 'ARHGAP4')},
 (72761, 72963): {('U52112.4-017', 'ARHGAP4'), ('U52112.4-019', 'ARHGAP4')},
 (72521, 72683): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (72253, 72315): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-

A partire dalla lista `cds_gtf_rows` costruire il dizionario `frame_dict` che permetterà di accedere, data una *feature* di tipo `CDS`, ai valori di *frame* in relazione a tutti i trascritti per cui la *feature* è un frammento di coding sequence.

Struttura del dizionario `frame_dict`:

- *chiave*: tupla *(start, end)* che rappresenta una *feature* di tipo `CDS`
- *valore*: dizionario annidato:
    - *chiave*: ID del gene (hugo name)
    - *valore*: dizionario annidato:
        - *chiave*: ID del trascritto
        - *valore*: frame
    
Inizializzazione del dizionario vuoto.

In [26]:
frame_dict = dict()

Attraversare la lista `cds_gtf_rows` per riempire il dizionario.

In [27]:
for row in cds_gtf_rows:
    (transcript_id, gene_id) = get_transcript_and_gene(row)
    cds_start = int(row.rstrip().split('\t')[3])
    cds_end = int(row.rstrip().split('\t')[4])
    frame = int(row.rstrip().split('\t')[7])
    
    inner_dict1 = frame_dict.get((cds_start, cds_end), dict())
    inner_dict2 = inner_dict1.get(gene_id, dict())
    inner_dict2.update([(transcript_id, frame)])
    inner_dict1.update([(gene_id, inner_dict2)])
    frame_dict.update([((cds_start, cds_end), inner_dict1)])

In [28]:
frame_dict

{(71783, 71788): {'ARHGAP4': {'U52112.4-005': 0}},
 (70312,
  70440): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-020': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-011': 0}},
 (69989,
  70210): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-011': 0}},
 (64935,
  65036): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-024': 0,
   'U52112.4-011': 0}},
 (64566, 64673): {'ARHGAP4': {'U52112.4-005': 0}},
 (64385, 64459): {'ARHGAP4': {'U52112.4-005': 0}},
 (72761, 72963): {'ARHGAP4': {'U52112.4-019': 0, 'U52112.4-017': 0}},
 (72521,
  72683): {'ARHGAP4': {'U52112.4-019': 1,
   'U52112.4-017': 1,
   'U52112.4-003': 1,
   'U52112.4-001': 1,
   'U52112.4-024': 1,
   'U52112.4-011': 1}},
 (72253,
  72315): {'ARHGAP4': {'U52112.4-019': 0,
   'U52112.4-017': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-024': 0}},
 (71872, 71965): {'ARHGAP4': {'U52112.4-019': 0}},
 (71865, 71965): {'AR

Le chiavi del dizionario `cds_inclusion_dict` che non compaiono in `exon_inclusion_dict` rappresentano *features* di tipo `CDS` che non coprono completamente un esone, e di conseguenza sono da scartare.

Cancellare quindi tali chiavi dal dizionario `cds_inclusion_dict`.

In [29]:
key_to_discard = set(cds_inclusion_dict).difference(set(exon_inclusion_dict))

u_list = [cds_inclusion_dict.pop(key) for key in key_to_discard]

A questo punto, data una chiave *(start, end)* in `exon_inclusion_dict`, si ha che il corrispondente valore è il set dei trascritti che includono l'esone *(start, end)*. Alla stessa chiave in `cds_inclusion_dict` (se esiste) corrisponderà come valore il set dei trascritti per cui la *feature* di tipo `CDS` *(start, end)* è il frammento di coding sequence che copre completamente l'esone *(start, end)*.
L'intersezione tra questi due set fornisce il set dei trascritti per cui l'esone *(start, end)* è coperto completamente da coding sequence.

Aggiornare quindi, per ogni chiave *(start, end)*, i valori in `cds_inclusion_dict` con il risultato dell'intersezione tra il set corrispondente alla chiave *(start, end)* in `cds_inclusion_dict` e il set corrispondente alla stessa chiave in `exon_inclusion_dict`.

In [30]:
for key in cds_inclusion_dict:
    inters = exon_inclusion_dict[key].intersection(cds_inclusion_dict[key])
    cds_inclusion_dict.update([(key, inters)])

In [31]:
cds_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005', 'ARHGAP4')},
 (64385, 64459): {('U52112.4-005', 'ARHGAP4')},
 (72521, 72683): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (72253, 72315): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4'

Le chiavi del dizionario `cds_inclusion_dict` forniscono ora tutte le tuple *(start, end)* che rappresentano esoni coperti completamente da coding sequence, e i rispettivi valori sono i set dei trascritti per cui l'esone *(start, end)* è coperto completamente da coding sequence.

Attraversare quindi il dizionario `cds_inclusion_dict` per costruire la lista di output `exon_coverage_list`.

In [34]:
exon_coverage_list = []

for feature in cds_inclusion_dict:
    for transcript_tuple in cds_inclusion_dict[feature]:
        transcript_id = transcript_tuple[0]
        gene_id = transcript_tuple[1]
        
        frame = frame_dict[feature][gene_id][transcript_id]
        strand = strand_dict[gene_id]
        
        feature_sequence = reverse_complement_in_case(genomic_reference[feature[0]-1:feature[1]], strand)
        exon_coverage_list.append((transcript_id, gene_id, feature[0], feature[1], codon_splitting(feature_sequence, frame)))

In [35]:
exon_coverage_list

[('U52112.4-005', 'ARHGAP4', 71783, 71788, 'gag aag'),
 ('U52112.4-001',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-005',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-020',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-003',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-011',
  'ARHGAP4',
  7031